In [1]:
from CircleNet.execution import get_parameters,Write_csv, save_agents
from CircleNet.animation import format_time, Displayer
from CircleNet.shanghai_platform_extension import Driver, Passenger, extract_result_data, create_simulation


#TODO

#problem of results?
#change name to value of time
#finish beautiful file


In [2]:
#definition of savers :

to_save=['id_sim', 'nb_passenger','nb_driver','execution_time', 'nb_match','driver_efficiency', 'passenger_efficiency',  
         'average_waiting_time','average_vks', 'total_vks']
result_file=Write_csv("data/general2.csv",to_save)

info_p=["id_number","Ox","Oy","Dx","Dy","last_departure_time","matched","driver","waiting"]
option_p={"position":("Ox","Oy"),"destination":("Dx","Dy")}

info_d=["id_number","Ox","Oy","Dx","Dy","departure_t","value_of_time","repetition_time","watches","viewed_announces","matched","passenger","vks","detour"]
option_d={"position":("Ox","Oy"),"destination":("Dx","Dy"),"time_perception":"value_of_time","departure_window":(_,"departure_t")}

files=[dict(name="drivers",selection_function=lambda x:isinstance(x,Driver),info_list=info_d,options_list=option_d),
       dict(name="passengers",selection_function=lambda x:isinstance(x,Passenger),info_list=info_p,options_list=option_p)]

def f(u):
    return "Simu n'{sim_number}\n{variable_parameters}\nTime: "+format_time(float(u))





#execution
import config as parameter_file
d=Displayer(text='Waiting...',function_when_display=f,optional_dictionnary=dict(sim_number=0,variable_parameters=""))
d.start()

for PARAMETERS,message in get_parameters(parameter_file):
    d.optional_dictionnary["sim_number"]+=1
    d.optional_dictionnary["variable_parameters"]=message#"D="+str(PARAMETERS["N_driver"])+" P="+str(PARAMETERS["N_passenger"])
    #construction
    simu=create_simulation(**PARAMETERS)
    simu.set_action(lambda x: d.change(x.time))
    #execution
    simu()
    results=extract_result_data(simu)
    #saving
    result_file.write(**results)
    save_agents(simu,*files)
    
d.quit()